# Import Necessary libraries

In [1]:
import pandas as pd 
import numpy as np 
import os 
#import xlrd 
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate 

# Setting OpenApi key

In [ ]:
os.environ["OPENAI_API_KEY"] ="your api key"
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.3)

C:\Users\akhil\AppData\Local\Temp\ipykernel_21772\966863086.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.3)


In [3]:
!pip install openpyxl



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Load dataset

In [7]:
import pandas as pd

file_path = r"C:\\Users\\akhil\\Downloads\\company_addresses_hindi.xlsx"

# Read the Excel file using openpyxl
df = pd.read_excel(file_path, engine="openpyxl")

# Display first few rows
print(df.head())


  कंपनी का नाम                                        सड़क का नाम  स्थान  \
0       विप्रो                         डोड्डाकनेल्ली, सरजापुर रोड    NaN   
1       विप्रो  प्लॉट नंबर 2, फेज 1, राजीव गांधी इन्फोटेक पार्...    NaN   
2       विप्रो                203/1, मनिकोंडा गांव, गाचीबोवली SEZ    NaN   
3       टीसीएस           आईटी/आईटीईएस एसईजेड, राजारहाट, न्यू टाउन    NaN   
4       टीसीएस               थिंक कैंपस, इलेक्ट्रॉनिक सिटी फेज II    NaN   

        शहर  पिनकोड  
0  बेंगलुरु  560035  
1      पुणे  411057  
2  हैदराबाद  500032  
3   कोलकाता  700160  
4  बेंगलुरु  560100  


In [9]:
df.shape

(15, 5)

In [11]:
print(df.columns)


Index(['कंपनी का नाम', 'सड़क का नाम', 'स्थान', 'शहर', 'पिनकोड'], dtype='object')


#  Define system message 

In [13]:
from langchain.schema import SystemMessage, HumanMessage

def get_area(street, city):
    """"Use OPENAI to predict the missing area."""
    address = f"{street},{city}"
    messages = [
        SystemMessage(content="You are an AI assistant that extracts area information form Hindi addresses."),
        HumanMessage(content=f"What is the area in the following address? {address}\n return only with area name.")
    ]
    response = llm(messages)
    return response.content.strip() if response.content else ""

In [15]:
import pandas as pd

# Dictionary to store already fetched locations
cache = {}

def get_area_with_cache(street, city):
    key = f"{street}_{city}"
    if key in cache:
        return cache[key]  # Return stored result
    try:
        result = get_area(street, city)  # API Call
        cache[key] = result  # Store result
        return result
    except Exception as e:
        print(f"API Error: {e}")
        return None  # Return None if API fails

df["स्थान"] = df.apply(lambda row: get_area_with_cache(row["सड़क का नाम"], row["शहर"]) if pd.isna(row["स्थान"]) else row["स्थान"], axis=1)


C:\Users\akhil\AppData\Local\Temp\ipykernel_21772\1248539136.py:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


In [16]:
df.head()

,कंपनी का नाम,सड़क का नाम,स्थान,शहर,पिनकोड
0,विप्रो,"डोड्डाकनेल्ली, सरजापुर रोड",सरजापुर,बेंगलुरु,560035
1,विप्रो,"प्लॉट नंबर 2, फेज 1, राजीव गांधी इन्फोटेक पार्...",हिंजवाड़ी,पुणे,411057
2,विप्रो,"203/1, मनिकोंडा गांव, गाचीबोवली SEZ",मनिकोंडा गांव,हैदराबाद,500032
3,टीसीएस,"आईटी/आईटीईएस एसईजेड, राजारहाट, न्यू टाउन",राजारहाट,कोलकाता,700160
4,टीसीएस,"थिंक कैंपस, इलेक्ट्रॉनिक सिटी फेज II",इलेक्ट्रॉनिक सिटी फेज II,बेंगलुरु,560100


In [17]:
df["स्थान"].head(15)

0                                  सरजापुर
1                                हिंजवाड़ी
2                            मनिकोंडा गांव
3                                 राजारहाट
4                 इलेक्ट्रॉनिक सिटी फेज II
5                                  माधापुर
6     सॉल्ट लेक इलेक्ट्रॉनिक्स कॉम्प्लेक्स
7                            मनिकोंडा गांव
8                न्यू टाउन-एक्शन एरिया-III
9                             शोलिंगनल्लूर
10                               हिंजवाड़ी
11                                 माधापुर
12                                अल्वरपेट
13                               हिंजवाड़ी
14                                 नागवारा
Name: स्थान, dtype: object

In [18]:
if "राज्य" not in df.columns:
    df["राज्य"] = None

# Function to get  state details

In [19]:
def get_state(pincode, street, city):
    """Use OpenAI to predict the state based on pincode, street, city"""
    address1 = f"{pincode},{street},{city}"
    messages = [
        SystemMessage(content="You are an AI assistant that extracts state information from Hindi addresses."),
        HumanMessage(content=f"What is the state in the following address? {address1}\n return only with state name in Hindi.")
    ]
    response = llm(messages)
    return response.content.strip() if response.content else ""

In [20]:
import time

def get_state_with_delay(pincode, street, city):
    time.sleep(1)  # Wait 1 second before making a request
    return get_state(pincode, street, city)

df["राज्य"] = df.apply(lambda row: get_state_with_delay(row["पिनकोड"], row["सड़क का नाम"], row["शहर"]) if pd.isna(row["राज्य"]) else row["राज्य"], axis=1)


In [21]:
df.head()

,कंपनी का नाम,सड़क का नाम,स्थान,शहर,पिनकोड,राज्य
0,विप्रो,"डोड्डाकनेल्ली, सरजापुर रोड",सरजापुर,बेंगलुरु,560035,कर्नाटक
1,विप्रो,"प्लॉट नंबर 2, फेज 1, राजीव गांधी इन्फोटेक पार्...",हिंजवाड़ी,पुणे,411057,महाराष्ट्र
2,विप्रो,"203/1, मनिकोंडा गांव, गाचीबोवली SEZ",मनिकोंडा गांव,हैदराबाद,500032,तेलंगाना
3,टीसीएस,"आईटी/आईटीईएस एसईजेड, राजारहाट, न्यू टाउन",राजारहाट,कोलकाता,700160,पश्चिम बंगाल
4,टीसीएस,"थिंक कैंपस, इलेक्ट्रॉनिक सिटी फेज II",इलेक्ट्रॉनिक सिटी फेज II,बेंगलुरु,560100,कर्नाटक


In [22]:
df['राज्य'].head(15)

0          कर्नाटक
1       महाराष्ट्र
2         तेलंगाना
3     पश्चिम बंगाल
4          कर्नाटक
5         तेलंगाना
6     पश्चिम बंगाल
7         तेलंगाना
8     पश्चिम बंगाल
9         तमिलनाडु
10      महाराष्ट्र
11        तेलंगाना
12        तमिलनाडु
13      महाराष्ट्र
14         कर्नाटक
Name: राज्य, dtype: object

In [23]:
if "देश" not in df.columns:
    df["देश"] = None

# Function to get country details

In [24]:
def get_country(state, city, pincode):
    """Use OpenAI to predict the country based on state, city, pincode."""
    address2 = f"{state},{city}, {pincode}"
    messages = [
        SystemMessage(content="You are an AI assistant that extracts country information from Hindi addresses."),
        HumanMessage(content=f"What is the country in the following address? {address2}\n return only with country name in Hindi.")
    ]

    response = llm(messages)
    return response.content.strip() if response.content else ""

In [25]:
df["देश"] = df.apply (lambda row: get_country(row["पिनकोड"], row["राज्य"], row["शहर"]) if pd.isna(row["देश"]) else row ["देश"], axis = 1)

In [26]:
df.head()

,कंपनी का नाम,सड़क का नाम,स्थान,शहर,पिनकोड,राज्य,देश
0,विप्रो,"डोड्डाकनेल्ली, सरजापुर रोड",सरजापुर,बेंगलुरु,560035,कर्नाटक,भारत
1,विप्रो,"प्लॉट नंबर 2, फेज 1, राजीव गांधी इन्फोटेक पार्...",हिंजवाड़ी,पुणे,411057,महाराष्ट्र,भारत
2,विप्रो,"203/1, मनिकोंडा गांव, गाचीबोवली SEZ",मनिकोंडा गांव,हैदराबाद,500032,तेलंगाना,भारत
3,टीसीएस,"आईटी/आईटीईएस एसईजेड, राजारहाट, न्यू टाउन",राजारहाट,कोलकाता,700160,पश्चिम बंगाल,भारत
4,टीसीएस,"थिंक कैंपस, इलेक्ट्रॉनिक सिटी फेज II",इलेक्ट्रॉनिक सिटी फेज II,बेंगलुरु,560100,कर्नाटक,भारत


In [32]:
df['देश'].head(15)

0     भारत
1     भारत
2     भारत
3     भारत
4     भारत
5     भारत
6     भारत
7     भारत
8     भारत
9     भारत
10    भारत
11    भारत
12    भारत
13    भारत
14    भारत
Name: देश, dtype: object

# Save updated dataset

In [38]:
df.to_excel("C:\\Users\\akhil\\Downloads\\company_addresses_hindi.xlsx", index=False)